In [1]:
import sys
sys.path.append("..")
from azure.cosmos import CosmosClient
from shared_utilities import helpers

C:\Users\patrick.tinsley\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Change fuzzy match to sql join

# Do I even need fuzzy matching ?????


In [3]:
conn_string = helpers.read_json("../local.settings.json")["Values"]["cosmosdb_deets"]
storageClient = CosmosClient.from_connection_string(conn_str=conn_string)

In [4]:
next_saturday, next_sunday = helpers.find_next_weekend()
# change second input '{next_saturday} and {next_sunday}' in the query below.


# get Betfair 
query = f"SELECT * FROM c where STARTSWITH(c.openDate, '2024-02-17') or STARTSWITH(c.openDate, '2024-02-18')" 
betfair_list = []
database = storageClient.get_database_client("fightstore")
betfair_mma_events = database.get_container_client("betfair_mma_events")
cosmos_bf = betfair_mma_events.query_items(query,enable_cross_partition_query= True)
for item in cosmos_bf:
    betfair_list.append(item)
         
# get Cards            
query = f"SELECT * FROM c where STARTSWITH(c.cardDate, '2024-02-17') or STARTSWITH(c.cardDate, '2024-02-18')"
cards=[]
fc_mma_cards = database.get_container_client("fc_mma_cards")
cosmos_fc = fc_mma_cards.query_items(query,enable_cross_partition_query = True)
for idc, card in enumerate(cosmos_fc):
    for idf, fight in enumerate(card["fights"]):
        card["fights"][idf]["index"]=idf # add the index to the specific fight object for patch later
        
    cards.append(card)


In [5]:
cards

[{'title': 'UFC 298',
  'date': 'Saturday, February 17,  6:30 PM ET',
  'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
  'fights': [{'main': True,
    'weight': '145',
    'fighterA': {'name': 'Alexander Volkanovski',
     'record': '26-3-0',
     'country': 'https://www.tapology.com/assets/flags/AU-429ed070b76f87de04728ef13b9d68e308be36cf4444bb32eb97564ae7016ff1.gif',
     'picture': 'https://images.tapology.com/headshot_images/41705/icon/alexander-volkanovski.jpg?1567271247',
     'link': 'https://www.tapology.com/fightcenter/fighters/41705-alex-volkanski'},
    'fighterB': {'name': 'Ilia Topuria',
     'record': '14-0-0',
     'country': 'https://www.tapology.com/assets/flags/GE-46e71ebd13f4e3f14b56f61538a1d026361bee9e3a68ed169cf30520d087c2fa.gif',
     'picture': 'https://images.tapology.com/headshot_images/129278/icon/Topuria-Hero.jpg?1659102670',
     'link': 'https://www.tapology.com/fightcenter/fighters/129278-ilia-topuria'},
    'fight_name': 'Alexander 

In [6]:
matched_cards, not_on_betfair = helpers.fuzzy_match_dict_list(cards, betfair_list, "fight_name" ,"name")

In [7]:
matched_cards # log these 

[{'title': 'UFC 298',
  'date': 'Saturday, February 17,  6:30 PM ET',
  'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
  'updatedAt': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
  'id': '7840535991427417239',
  'cardDate': '2024-02-17',
  'partition': '2443883002582277486',
  'fights': [{'main': True,
    'weight': '145',
    'fighterA': {'name': 'Alexander Volkanovski',
     'record': '26-3-0',
     'country': 'https://www.tapology.com/assets/flags/AU-429ed070b76f87de04728ef13b9d68e308be36cf4444bb32eb97564ae7016ff1.gif',
     'picture': 'https://images.tapology.com/headshot_images/41705/icon/alexander-volkanovski.jpg?1567271247',
     'link': 'https://www.tapology.com/fightcenter/fighters/41705-alex-volkanski'},
    'fighterB': {'name': 'Ilia Topuria',
     'record': '14-0-0',
     'country': 'https://www.tapology.com/assets/flags/GE-46e71ebd13f4e3f14b56f61538a1d026361bee9e3a68ed169cf30520d087c2fa.gif',
     'picture': 'https://images.tapology.

In [24]:
not_on_betfair # -> These are fights in the card which we can't find in betfair

[{'main': True,
  'weight': '185',
  'fighterA': {'name': 'Robert Whittaker',
   'record': '24-7-0',
   'country': 'https://www.tapology.com/assets/flags/AU-429ed070b76f87de04728ef13b9d68e308be36cf4444bb32eb97564ae7016ff1.gif',
   'picture': 'https://images.tapology.com/headshot_images/17398/icon/Robert-Whittaker.jpg?1488681693',
   'link': 'https://www.tapology.com/fightcenter/fighters/17398-robert-whittaker'},
  'fighterB': {'name': 'Paulo Costa',
   'record': '14-2-0',
   'country': 'https://www.tapology.com/assets/flags/BR-45af1ab77cd750eff617a8b71f64b318c9a4ecf06c863a2c63cff71550930fe7.gif',
   'picture': 'https://images.tapology.com/headshot_images/64306/icon/Screen_Shot_2020-12-02_at_5.46.51_AM.png?1606898823',
   'link': 'https://www.tapology.com/fightcenter/fighters/64306-paulo-henrique-costa-borrachinha'},
  'fight_name': 'Robert Whittaker v Paulo Costa',
  'fight_id': '8758264360754347100',
  'index': 1},
 {'main': True,
  'weight': '170',
  'fighterA': {'name': 'Geoff Neal'

In [10]:
# not going to work because it removes all over fights.....

for card in matched_cards:
    # print(card)
    for fight in card["fights"]:
        # print(fight)
        # print(fight["index"])
        operations =[
            { "op": "add", "path": f"/fights/{str(fight["index"])}/betfair_event_id", "value": fight["betfair_event_id"] },
            { "op": "add", "path": f"/fights/{str(fight["index"])}/betfair_country_code", "value": fight["betfair_country_code"] },
            { "op": "add", "path": f"/fights/{str(fight["index"])}/betfair_open_date", "value": fight["betfair_open_date"] },
            { "op": "add", "path": f"/fights/{str(fight["index"])}/betfair_timezone", "value": fight["betfair_timezone"] }
        ]
        
        # response = fc_mma_cards.patch_item(item=card["id"], partition_key=card["link"], patch_operations=operations)

 
    # only upsert because fight will have existed beforehand.
    
    # no need to pure create because it will have been created from ingestion
    # no need to patch because this will be the truth now..

0
1
2
3
4
5
6
7
8
9
10
11


In [12]:
"NOT ON BETFAIR: "+ str(not_on_betfair)

'NOT ON BETFAIR: []'

In [68]:
response # log this.

[{'op': 'add', 'path': '/fights/0/betfair_event_id', 'value': '32834431'},
 {'op': 'add', 'path': '/fights/0/betfair_country_code', 'value': 'GB'},
 {'op': 'add',
  'path': '/fights/0/betfair_open_date',
  'value': '2024-02-18T05:00:00.000Z'},
 {'op': 'add', 'path': '/fights/0/betfair_timezone', 'value': 'GMT'}]

In [66]:
# ops = [{ "op": "add", "path": "/fights/0/test", "value": "patrick"}]
# response = fc_mma_cards.patch_item(item="4797802081340855333", partition_key="https://www.tapology.com/fightcenter/events/107060-ufc-298", patch_operations=ops)


{'title': 'UFC 298',
 'date': 'Saturday, February 17,  6:30 PM ET',
 'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
 'fights': [{'main': True,
   'weight': '145',
   'fighterA': {'name': 'Alexander Volkanovski',
    'record': '26-3-0',
    'country': 'https://www.tapology.com/assets/flags/AU-429ed070b76f87de04728ef13b9d68e308be36cf4444bb32eb97564ae7016ff1.gif',
    'picture': 'https://images.tapology.com/headshot_images/41705/icon/alexander-volkanovski.jpg?1567271247',
    'link': 'https://www.tapology.com/fightcenter/fighters/41705-alex-volkanski'},
   'fighterB': {'name': 'Ilia Topuria',
    'record': '14-0-0',
    'country': 'https://www.tapology.com/assets/flags/GE-46e71ebd13f4e3f14b56f61538a1d026361bee9e3a68ed169cf30520d087c2fa.gif',
    'picture': 'https://images.tapology.com/headshot_images/129278/icon/Topuria-Hero.jpg?1659102670',
    'link': 'https://www.tapology.com/fightcenter/fighters/129278-ilia-topuria'},
   'fight_name': 'Alexander Volkanovski v Il

In [ ]:
# Work out where to store this. 
# schema enforcement.
# make date more reliable.
# automate get bf api session token
# cosmos data model
    # how to add specific schemas.

In [37]:
import json
operations

[{'op': 'add', 'path': 'fights/0/betfair_event_id', 'value': '32834431'},
 {'op': 'add', 'path': 'fights/0/betfair_country_code', 'value': 'GB'},
 {'op': 'add',
  'path': 'fights/0/betfair_open_date',
  'value': '2024-02-18T05:00:00.000Z'},
 {'op': 'add', 'path': 'fights/0/betfair_timezone', 'value': 'GMT'}]